In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import os

In [22]:
import numpy as np
import os

def load_pcd_ascii(file_path, num_points=1024):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Encontrar la línea que indica el comienzo de los datos
    data_start_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("DATA"):
            data_start_idx = i + 1
            break
    
    if data_start_idx is None:
        raise ValueError("No se encontró la sección de datos en el archivo PCD.")
    
    # Leer las líneas de datos y convertirlas en puntos
    points = []
    for line in lines[data_start_idx:]:
        if line.strip():  # Ignorar líneas vacías
            point = list(map(float, line.strip().split()))
            if len(point) == 3:  # Solo procesar puntos válidos con x, y, z
                points.append(point)
    
    # Convertir la lista de puntos a un array de numpy
    points = np.array(points, dtype=np.float32)

    # Ajustar el número de puntos
    if points.shape[0] > num_points:
        points = points[:num_points, :]  # Truncar si hay más de num_points
    elif points.shape[0] < num_points:
        # Padding con ceros si hay menos de num_points
        padding = np.zeros((num_points - points.shape[0], 3), dtype=np.float32)
        points = np.vstack((points, padding))
    
    return points

file_path = "/home/cdonoso/ros2_ws/src/ros2_pcl_segmentation/pcl_car_segmentation/car_dataset_clouds"
files_paths = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith(".pcd")]

data = np.array([load_pcd_ascii(f) for f in files_paths])
labels = np.ones(len(data), dtype=np.int32)


In [3]:

train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


In [30]:
class PointNet(nn.Module):
    def __init__(self, num_points=1024, num_classes=2):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=1)
        self.conv3 = nn.Conv1d(128, 1024, kernel_size=1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.transpose(1, 2)  # Transponer para PyTorch (B, 3, num_points)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return self.softmax(x)

# Crear el modelo
model = PointNet(num_points=1024, num_classes=2)


In [31]:
train_data = torch.tensor(train_data, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)

# Crear un dataset y dataloader
dataset = TensorDataset(train_data, train_labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

/tmp/ipykernel_7121/1002633746.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor(train_data, dtype=torch.float32)
/tmp/ipykernel_7121/1002633746.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels, dtype=torch.long)


In [32]:
# Configuración de entrenamiento
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 10

# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    
    # Validación en cada época
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for data, labels in val_loader:
            output = model(data)
            val_loss += criterion(output, labels).item()
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}%")


Epoch 1/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 2/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 3/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 4/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 5/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 6/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 7/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 8/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 9/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%
Epoch 10/10, Loss: 0.31326165795326233, Val Loss: 0.9397850036621094, Val Accuracy: 100.0%


In [34]:
# Exportar a ONNX
dummy_input = torch.randn(1, 3, 1024)  # Dimensiones de entrada
model_path = "/home/cdonoso/ros2_ws/src/ros2_pcl_segmentation/pcl_car_segmentation/model/"
torch.onnx.export(model, dummy_input, f"{model_path}pointnet_model.onnx", opset_version=13)


RuntimeError: Given groups=1, weight of size [64, 3, 1], expected input[1, 1024, 3] to have 3 channels, but got 1024 channels instead